In [3]:
import tkinter
import psycopg2
from tkinter import messagebox, Label, Entry, Button, Frame, LabelFrame, ttk, Spinbox, StringVar, Toplevel
import pandas as pd
import datetime
import numpy as np
#373957676814

#CREATE EXTENSION pgcrypto;
INSERT INTO users (email, password) VALUES (
  'johndoe@mail.com',
  crypt('johnspassword', gen_salt('bf'))
);

SELECT id 
  FROM users
 WHERE email = 'johndoe@mail.com' 
   AND password = crypt('wrongpassword', password);

In [1]:
master = tkinter.Tk()
master.title("A simple GUI")
master.geometry("300x300")
master.resizable(0,0)

frame1 = tkinter.LabelFrame(master,text="Login")
#frame1.place(relx=0.5, rely=0.5, anchor='c')
frame2 = tkinter.LabelFrame(master,text="Signup")
frame2.place(relx=0.5, rely=0.5, anchor='c')
frame3 = tkinter.LabelFrame(master,text="Welcome")

def signup_frame():
    frame1.place_forget()
    frame2.place(relx=0.5, rely=0.5, anchor='c')
    
def login_frame():
    frame2.place_forget()
    frame1.place(relx=0.5, rely=0.5, anchor='c')
    
def acc_success():

        if len((fn.get()) and (ln.get()) and (un.get()) and (upass.get()) and upassc.get()) == 0:
            messagebox.showerror("error","please enter")
        elif (upass.get() != upassc.get()):
            messagebox.showerror("error","password mismatch")
        else:
            conn = None
            try:
                # connect to the PostgreSQL server
                #print('Connecting to the PostgreSQL database...')
                conn = psycopg2.connect(
                host="salesdb.cah6zhd5h1in.ap-south-1.rds.amazonaws.com",
                database="postgres",
                user="postgres",
                password="postgresadmin")
                # create a cursor
                cur = conn.cursor()
                cur.execute(f"SELECT User_Name FROM EMPLOGIN WHERE User_Name = '{un.get()}'")
                dbun = cur.fetchone()
                if dbun !=None:
                    messagebox.showerror("error","username already exists")

                else:
                    
                    cur.execute(f"""INSERT INTO 
            JOB(Job_ID)
                VALUES
            (concat('JO', nextval('job_id_seq')))""")
                    cur.execute("SELECT currval('job_id_seq')")
                    curjobid = cur.fetchone()
                    print(curjobid[0])
                    cur.execute(f"""INSERT INTO 
            EMPLOYEE(Employee_ID, First_Name, Last_Name,Job_ID)
                VALUES
            (concat('EMP', nextval('employee_id_seq')),'{fn.get()}','{ln.get()}','JO{curjobid[0]}')""")
                    cur.execute("SELECT currval('employee_id_seq')")
                    curemid = cur.fetchone()
                    print(curemid)
                    cur.execute(f"""INSERT INTO 
            EMPLOGIN(User_Name, Password, Employee_ID)
                VALUES
            ('{un.get()}',crypt('{upass.get()}', gen_salt('bf')),'EMP{curemid[0]}')""")
                    messagebox.showinfo('success',"Employee created successfully")
                conn.commit()
                fn.delete(0,'end')
                ln.delete(0,'end')
                un.delete(0,'end')
                upass.delete(0,'end')
                upassc.delete(0,'end')
                body = '{"body":"Hello from postgres!"}'
                
                cur.execute(f"SELECT * from aws_lambda.invoke(aws_commons.create_lambda_function_arn('arn:aws:lambda:ap-south-1:373957676814:function:rdstosnsfunc', 'ap-south-1'), '{body}'::json )")

                # close the communication with the PostgreSQL
                cur.close()
            except (Exception, psycopg2.DatabaseError) as error:
                print(error)
            finally:
                if conn is not None:
                    conn.close()
                    #print('Database connection closed.')
        
def acc_login():
    
        if len((log_un.get()) and (log_ps.get())) == 0:
            messagebox.showerror("error","please enter")
        else:
            conn = None
            try:
                # connect to the PostgreSQL server
                #print('Connecting to the PostgreSQL database...')
                conn = psycopg2.connect(
                        host="localhost",
                        database="myinner_db",
                        user="mynonsuperuser",
                        password="nonp")
                # create a cursor
                cur = conn.cursor()
                cur.execute(f"SELECT Employee_ID FROM EMPLOGIN WHERE User_Name = '{log_un.get()}' and Password = '{log_ps.get()}'")
                lid = cur.fetchone()
                
                if lid == None:
                    messagebox.showerror("error","username or password incorrect")
                else:
                    messagebox.showinfo("error","login success")                 
                    # close the communication with the PostgreSQL
                    cur.close()
                    loggedin()
            except (Exception, psycopg2.DatabaseError) as error:
                print(error)
            finally:
                if conn is not None:
                    conn.close()
                    #print('Database connection closed.')
        
def loggedin():
    frame1.place_forget()
    frame2.place_forget()
    frame3.place(relx=0.5, rely=0.5, anchor='c')
    print("logged")

tkinter.Label(frame1,text="username").grid(row=0,column=0)
log_un = tkinter.Entry(frame1)
log_un.grid(row=0,column=1)
tkinter.Label(frame1,text="password").grid(row=1,column=0)
log_ps = tkinter.Entry(frame1,show='*')
log_ps.grid(row=1,column=1)
tkinter.Button(frame1,text="login",command=acc_login).grid(row=2,column=1)
tkinter.Label(frame1,text="Already having account!!").grid(row=3,column=1)
tkinter.Button(frame1,text="Signup",command=signup_frame).grid(row=4,column=1)


tkinter.Label(frame2,text="FirstName").grid(row=0,column=0)
fn = tkinter.Entry(frame2)
fn.grid(row=0,column=1)
tkinter.Label(frame2,text="LastName").grid(row=1,column=0)
ln = tkinter.Entry(frame2)
ln.grid(row=1,column=1)
tkinter.Label(frame2,text="username").grid(row=2,column=0)
un = tkinter.Entry(frame2)
un.grid(row=2,column=1)
tkinter.Label(frame2,text="Password").grid(row=3,column=0)
upass = tkinter.Entry(frame2,show='*')
upass.grid(row=3,column=1)
tkinter.Label(frame2,text="Confirm password").grid(row=4,column=0)
upassc = tkinter.Entry(frame2,show='*')
upassc.grid(row=4,column=1)
tkinter.Button(frame2,text="signup",command=acc_success).grid(row=5,column=1)
#tkinter.Label(frame2,text="New user!!").grid(row=6,column=1)
#tkinter.Button(frame2,text="login",command=login_frame).grid(row=7,column=1)


tkinter.Label(frame3,text="success").pack()

master.mainloop()

NameError: name 'tkinter' is not defined

In [4]:
main = tkinter.Tk()
main.title("A simple GUI")
main.geometry("600x600")
#main.resizable(0,0)



def acc_login():
    
    if len((Uname.get()) and (Upass.get())) == 0:
        messagebox.showerror("error","please enter")
    else:
        conn = None
        try:
                # connect to the PostgreSQL server
                #print('Connecting to the PostgreSQL database...')
            conn = psycopg2.connect(
                                host="salesdb.cah6zhd5h1in.ap-south-1.rds.amazonaws.com",
                                database="postgres",
                                user="postgres",
                                password="postgresadmin")
                # create a cursor
            cur = conn.cursor()
            cur.execute(f"SELECT Employee_ID FROM EMPLOGIN WHERE User_Name = '{Uname.get()}' and Password = crypt('{Upass.get()}', Password)")
            lid = cur.fetchone()
                
            if lid == None:
                messagebox.showerror("error","username or password incorrect")
            else:
                loginframe.pack_forget()
                mainframe.pack()
                messagebox.showinfo("error","login success")                 
            # close the communication with the PostgreSQL
                cur.close()
            #loggedin()
        except (Exception, psycopg2.DatabaseError) as error:
            print(error)
        finally:
            if conn is not None:
                conn.close()
                    #print('Database connection closed.')
        
loginframe = Frame(main)
loginframe.pack()

Label(loginframe,text="Username").grid(row=0,column=0)
Uname = Entry(loginframe)
Uname.grid(row=0,column=1)
Label(loginframe,text="password").grid(row=1,column=0)
Upass = Entry(loginframe,show='*')
Upass.grid(row=1,column=1)
Ubtn = Button(loginframe,text='login',command=acc_login)
Ubtn.grid(row=2,column=1)

mainframe = Frame(main)


dashboard_frame = LabelFrame(mainframe,text=" DashBoard ") 
dashboard_frame.pack(side="left",expand=True,fill='both')

product_frame = LabelFrame(mainframe,text=" Products ")
product_frame.pack(side="left",expand=True,fill='both')

sales_frame = Label(product_frame)
sales_frame.pack(side='bottom')

#Label(dashboard_frame,text="EmployeeID").grid(row=0,column=0)
#Label(dashboard_frame,text="EmployeID").grid(row=1,column=0)

#Label(product_frame,text="Productid").grid(row=0,column=0)
#Label(product_frame,text="product name").grid(row=1,column=0)
#Label(product_frame,text="Product price").grid(row=2,column=0)


Label(sales_frame,text="total amount",font="Helvitica 15 bold").grid(row=0,column=0)
totalentry = Entry(sales_frame)
totalentry.grid(row=0,column=1)
totalentry.insert(0,0)
totalentry.config(state='readonly')



def changedb():
    top = Toplevel(main)
    top.geometry("400x400")
    top.title("toplevel")
    top.resizable(0,0)
    
    def call(input):
        if input.isdigit():
            change.config(state='normal')
            change.delete(0,'end')
            #rint(input,totalentry.get())
            change.insert(0,(int(input)-int(totalentry.get())))
            change.config(state='readonly')
            return True
        elif input == "":
            change.delete(0,'end')
            change.insert(0,0)
            return True

        else:
            return False
    
    l2 = Label(top, text = "Customer Entry")
    l2.grid(row=0,column=0,columnspan=2)
    Label(top,text="Customer Name").grid(row=1,column=0)
    CName = Entry(top)
    CName.grid(row=1,column=1)
    Label(top,text="Customer Phno").grid(row=2,column=0)
    CPNO = Entry(top)
    CPNO.grid(row=2,column=1)
    Label(top,text="total amount:").grid(row=3,column=0)
    Label(top,text=totalentry.get()).grid(row=3,column=1)
    Label(top,text="amount taken").grid(row=4,column=0)
    


    regi = top.register(call)
    amounttaken = Entry(top)
    amounttaken.grid(row=4,column=1)
    amounttaken.config(validate ="key", 
             validatecommand =(regi, '%P'))

    Label(top,text="change given").grid(row=5,column=0)
    change = Entry(top)
    change.grid(row=5,column=1)
    change.config(state="readonly")
    
    def exit():
        #qty1.set(var.get())
        top.destroy()
        
    button = Button(top,text="ok",command=exit)
    button.grid(row=6,column=1)
    main.wait_window(top)
    
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(
                                host="salesdb.cah6zhd5h1in.ap-south-1.rds.amazonaws.com",
                                database="postgres",
                                user="postgres",
                                password="postgresadmin")
        # create a cursor
        cur = conn.cursor()
        proid = []

        # display the PostgreSQL database server version
        for child in tree_add.get_children():
            
            cur.execute(f"SELECT product_id from PRODUCT WHERE product_name = '{tree_add.item(child)['values'][0]}'")
            db = cur.fetchone()
            proid.append(db)
        
         
        for child in tree_add.get_children():
            for f in proid:
                cur.execute(f"SELECT stock from product where product_id = '{f[0]}'")
                qty = cur.fetchone()
                cur.execute(f"UPDATE product SET stock = '{abs(tree_add.item(child)['values'][3] - qty[0])}' WHERE product_id = '{f[0]}'")
            
        #cur.execute("UPDATE courses SET published_date = '2020-08-01' WHERE course_id = 3)
        print(proid)
        conn.commit()
        messagebox.showinfo("Success","purchase success")

        # close the communication with the PostgreSQL
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')
            
subbtn = Button(sales_frame,text="OK",command=changedb)
subbtn.grid(row=4,column=1)



tframe1 = Frame(product_frame)
tframe1.pack()
tframe2 = Frame(product_frame)
tframe2.pack()

treev = ttk.Treeview(tframe1, selectmode ='browse')
 
# Calling pack method w.r.to treeview
treev.pack(side ='right')
 
# Constructing vertical scrollbar
# with treeview
verscrlbar = ttk.Scrollbar(tframe1,
                           orient ="vertical",
                           command = treev.yview)
 
# Calling pack method w.r.to vertical
# scrollbar
verscrlbar.pack(side ='right', fill ='x')
 
# Configuring treeview
treev.configure(xscrollcommand = verscrlbar.set)


product_list = ["Product_Name","Price", "Category"]
treev["columns"] = ([f for f in product_list])    
for f in range(len(product_list)):
# Defining number of columns
    
    treev.column(f)
    treev.heading(f, text =product_list[f])

# Defining heading
treev['show'] = 'headings'


tree_add = ttk.Treeview(tframe2, selectmode ='browse')
 
# Calling pack method w.r.to treeview
tree_add.pack(side ='right')
 
# Constructing vertical scrollbar
# with treeview
verscrlbar = ttk.Scrollbar(tframe2,
                           orient ="vertical",
                           command = tree_add.yview)
 
# Calling pack method w.r.to vertical
# scrollbar
verscrlbar.pack(side ='right', fill ='x')
 
# Configuring treeview
tree_add.configure(xscrollcommand = verscrlbar.set)


final = ["Product_Name","Price", "Category","Quantity","Discount","DiscountedPrice","TotalPrice"]
tree_add["columns"] = ([f for f in final])    
for f in range(len(final)):
# Defining number of columns
    
    tree_add.column(f,width=100)
    tree_add.heading(f, text =final[f])

# Defining heading
tree_add['show'] = 'headings'
 
# Assigning the width and anchor to  the
# respective columns
#treev.column("1", width = 90, anchor ='c')
#treev.column("2", width = 90, anchor ='se')
#treev.column("3", width = 90, anchor ='se')
 
# Assigning the heading names to the
# respective columns
#treev.heading("1", text ="Name")
#treev.heading("2", text ="Sex")
#treev.heading("3", text ="Age")

conn = None
try:
    # connect to the PostgreSQL server
    print('Connecting to the PostgreSQL database...')
    conn = psycopg2.connect(
                                host="salesdb.cah6zhd5h1in.ap-south-1.rds.amazonaws.com",
                                database="postgres",
                                user="postgres",
                                password="postgresadmin")
        # create a cursor
    cur = conn.cursor()
        
        # display the PostgreSQL database server version
    cur.execute("SELECT * FROM product")
    product_values = cur.fetchall()
    cur.execute("SELECT * FROM discount")
    discounted = cur.fetchall()
    df = pd.DataFrame(product_values,columns =['product_id','product_name','description','stock','price','category','supplier_id'])
    df1 = pd.DataFrame(discounted,columns=["promotionid","startdate","enddate","productid","percentage"])
    joineddf = df.join(df1)
    joineddf['enddates'] = pd.to_datetime(joineddf['enddate']).dt.date
    joineddf['startdates'] = pd.to_datetime(joineddf['startdate']).dt.date
    today = datetime.datetime.today()
    joineddf.loc[(datetime.date(today.year,today.month,today.day) >= joineddf.startdates) & (datetime.date(today.year,today.month,today.day)<= joineddf.enddates), 'percentages'] = joineddf["percentage"]
    joineddf['percentages'] = joineddf['percentages'].fillna(0)
    
    #print(df)
    #print(product_values)
    conn.commit()

        # close the communication with the PostgreSQL
    cur.close()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        conn.close()
        print('Database connection closed.')

        
def callback(input):
    if input == "":
        treev.delete(*treev.get_children())
        return True
    else:
        treev.delete(*treev.get_children())
        for row in df[[input in x.lower() for x in df['product_name']]][["product_name","price","category"]].values.tolist():
            treev.insert("", 'end',values = row)
        return True

e = Entry(tframe1)
e.pack(side="right")
reg = main.register(callback)
  
e.config(validate ="key", 
         validatecommand =(reg, '%P'))
totallist = []


def getqty():
    def validate(user_input):
        if user_input.isdigit():
                #minval = int(main.nametowidget(spin).config('from')[4])
                #maxval = int(main.nametowidget(spin).config('to')[4])

                # check if the number is within the range
            if int(user_input) in range(1, 11):
                    #print ("Out of range")
                return True
            else:
                return False
        elif  user_input == "":
            return True

            # return false is input is not numeric
        else:
                #print("Not numeric")
            return False

   
    top = Toplevel(main)
    top.geometry("180x100")
    top.title("toplevel")
    top.resizable(0,0)
    l2 = Label(top, text = "This is toplevel window")
    l2.pack()
    var = StringVar()
    checkvar = StringVar()
    spin = Spinbox(top,textvariable=var, from_=1, to=10)
    spin.pack()

    range_validation = top.register(validate)

    spin.config(validate ="key",
             validatecommand = (range_validation, '%P'))
    #qty = var.get()
    #print(qty)
    def exit():
        qty1.set(var.get())
        top.destroy()
        
    button = Button(top,text="ok",command=exit)
    button.pack()
    main.wait_window(top)
    
    
qty1 = StringVar()
def add(event):
    
    toadd = []
    item = treev.focus()
    toadd.extend([f for f in treev.item(item,'values')])
    print(toadd)
    print(treev.item(item,'values'))
    prname = treev.item(item,'values')[0]
    print(prname)
    price = treev.item(item,'values')[1]
    print("price:",price)
    disc = joineddf['percentages'][joineddf['product_name'] == prname].to_string(index=False)
    getdis = f"0.{disc if disc == 10 else '05'}"
    print("getdis:",float(getdis))
    dis = int(price) - (float(getdis)*int(price))
    print("discountprice",dis)
    getqty()
    
    print("quantity",qty1.get())
    totalprice = round(dis) * int(qty1.get())
    print("totalprice",totalprice)
    #getl = [f for f in treev.item(item,'values')]
    #print(getl)
    toadd.extend([qty1.get(),getdis,dis,totalprice])
    #toadd.extend([treev.item(item,'values'),])
    print(toadd)
    tree_add.insert("", 'end',values = toadd)
    totallist.clear()
    #print(int(treev.item(item,'values')[1]))
    for child in tree_add.get_children():
        
        totallist.append(tree_add.item(child)["values"][6])
        print(totallist)
        print(sum(totallist))
        totalentry.config(state="normal")
        totalentry.delete(0,'end')
        totalentry.insert(0,sum(totallist))
        totalentry.config(state="readonly")
    #print("you clicked on", treev.item(item,'values'))
    
def delete(event):
    tree_add.delete(tree_add.get_children())
    

treev.bind("<Double-1>", add)
tree_add.bind("<Double-1>",delete)
main.mainloop()

Connecting to the PostgreSQL database...
Database connection closed.
['Bread', '40', 'Processed Food']
('Bread', '40', 'Processed Food')
Bread
price: 40
getdis: 0.05
discountprice 38.0
quantity 5
totalprice 190
['Bread', '40', 'Processed Food', '5', '0.05', 38.0, 190]
[190]
190
Connecting to the PostgreSQL database...
[('PRO21',)]
Database connection closed.


In [17]:
45 - (0.05 * 45)

42.75

In [279]:
10% of $45 = 0.10 × 45 = $4.50

$45 – $4.50 = $40.50

or

90% of $45 = 0.90 × 45 = $40.50

3


In [33]:
joineddf

,product_id,product_name,description,stock,price,category,supplier_id,promotionid,startdate,enddate,productid,percentage,enddates,startdates,percentages
0,PRO1,Mother Dairy Lassi,It is a perfect yogurt-based beverage for the ...,11,222,Dairy,SUP1,PROM6,2023-01-01,2023-02-01,PRO1,10,2023-02-01,2023-01-01,10
1,PRO2,Amul Premium Dahi,Dahi or curd is a delicious and healthy additi...,15,464,Dairy,SUP1,PROM7,2023-01-01,2023-02-01,PRO2,5,2023-02-01,2023-01-01,5
2,PRO3,Mother Dairy Paneer,"a sweet milky smell along with a fresh, natura...",16,233,Dairy,SUP1,PROM8,2023-01-01,2023-02-01,PRO3,5,2023-02-01,2023-01-01,5
3,PRO4,Mother Dairy Tadka Chach,refreshing taste and the flavorful green chili...,10,206,Dairy,SUP1,PROM9,2023-01-01,2023-02-01,PRO4,5,2023-02-01,2023-01-01,5
4,PRO5,Amul Cheese Slices,"homemade sandwiches, burgers or even a desi ma...",20,466,Dairy,SUP1,PROM10,2023-01-01,2023-02-01,PRO5,5,2023-02-01,2023-01-01,5
5,PRO6,Epigamia Artisanal Mishti Doi,Mishti Doi is a classic Bengali dessert that n...,16,88,Dairy,SUP1,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0
6,PRO7,Paper Boat Milkshakes,Paperboat’s milkshakes are available in three ...,12,432,Dairy,SUP1,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0
7,PRO8,Sleepy Owl Cold Coffee,the Sleepy owl Classic Cold Coffee for its per...,19,86,Dairy,SUP1,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0
8,PRO9,Epigamia Snack Pack,the fresh mango aroma upon opening the pack to...,17,81,Dairy,SUP1,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0
9,PRO10,Raw Pressery 18g Protein Milkshakes,The peppermint flavor in this protein milkshak...,14,333,Dairy,SUP1,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0


In [213]:
joineddf['enddates'] = pd.to_datetime(joineddf['enddate']).dt.date

In [248]:
joineddf.loc[(datetime.date(today.year,today.month,today.day) >= joineddf.startdates) & (datetime.date(today.year,today.month,today.day)<= joineddf.enddates), 'percentages'] = joineddf["percentage"]
joineddf['percentages'] = joineddf['percentages'].fillna(0)

In [41]:
#joineddf[["product_name","price","category","percentages"]]
f"0.{joineddf['percentages'][joineddf['product_name'] == 'Epigamia Artisanal Mishti Doi'].to_string(index=False)}"

'0.0'

'50%'

In [220]:


joineddf["discountedperc"] = np.where((joineddf['startdates']  <= datetime.date(today.year,today.month,today.day) <= joineddf["enddates"]),joineddf["percentage"], np.nan)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [214]:
joineddf

,product_id,product_name,description,stock,price,category,supplier_id,promotionid,startdate,enddate,productid,percentage,startdates,enddates
0,PRO1,Mother Dairy Lassi,It is a perfect yogurt-based beverage for the ...,11,222,Dairy,SUP1,PROM6,2023-01-01,2023-02-01,PRO1,10,2023-01-01,2023-02-01
1,PRO2,Amul Premium Dahi,Dahi or curd is a delicious and healthy additi...,15,464,Dairy,SUP1,PROM7,2023-01-01,2023-02-01,PRO2,5,2023-01-01,2023-02-01
2,PRO3,Mother Dairy Paneer,"a sweet milky smell along with a fresh, natura...",16,233,Dairy,SUP1,PROM8,2023-01-01,2023-02-01,PRO3,5,2023-01-01,2023-02-01
3,PRO4,Mother Dairy Tadka Chach,refreshing taste and the flavorful green chili...,10,206,Dairy,SUP1,PROM9,2023-01-01,2023-02-01,PRO4,5,2023-01-01,2023-02-01
4,PRO5,Amul Cheese Slices,"homemade sandwiches, burgers or even a desi ma...",20,466,Dairy,SUP1,PROM10,2023-01-01,2023-02-01,PRO5,5,2023-01-01,2023-02-01
5,PRO6,Epigamia Artisanal Mishti Doi,Mishti Doi is a classic Bengali dessert that n...,16,88,Dairy,SUP1,NaN,NaN,NaN,NaN,NaN,NaT,NaT
6,PRO7,Paper Boat Milkshakes,Paperboat’s milkshakes are available in three ...,12,432,Dairy,SUP1,NaN,NaN,NaN,NaN,NaN,NaT,NaT
7,PRO8,Sleepy Owl Cold Coffee,the Sleepy owl Classic Cold Coffee for its per...,19,86,Dairy,SUP1,NaN,NaN,NaN,NaN,NaN,NaT,NaT
8,PRO9,Epigamia Snack Pack,the fresh mango aroma upon opening the pack to...,17,81,Dairy,SUP1,NaN,NaN,NaN,NaN,NaN,NaT,NaT
9,PRO10,Raw Pressery 18g Protein Milkshakes,The peppermint flavor in this protein milkshak...,14,333,Dairy,SUP1,NaN,NaN,NaN,NaN,NaN,NaT,NaT


In [206]:

if datetime.date(2023,1,1) <= datetime.date(today.year,today.month,today.day) <= datetime.date(2023,2,1):
    print("in between")
else:
    print("No!")

in between


'12'

In [80]:
for f in df.index:
    pass
print(df[['d' in x.lower() for x in df['product_name']]].values.tolist())
#print(f)

[['PRO1', 'Mother Dairy Lassi', 'It is a perfect yogurt-based beverage for the hot sweltering summers.', 11, 222, 'Dairy', 'SUP1'], ['PRO2', 'Amul Premium Dahi', 'Dahi or curd is a delicious and healthy addition to your meal.', 15, 464, 'Dairy', 'SUP1'], ['PRO3', 'Mother Dairy Paneer', 'a sweet milky smell along with a fresh, natural taste.', 16, 233, 'Dairy', 'SUP1'], ['PRO4', 'Mother Dairy Tadka Chach', 'refreshing taste and the flavorful green chili kick at the end.', 10, 206, 'Dairy', 'SUP1'], ['PRO6', 'Epigamia Artisanal Mishti Doi', 'Mishti Doi is a classic Bengali dessert that never fails to impress.', 16, 88, 'Dairy', 'SUP1'], ['PRO8', 'Sleepy Owl Cold Coffee', 'the Sleepy owl Classic Cold Coffee for its perfectly balanced proportions of milk and water.', 19, 86, 'Dairy', 'SUP1'], ['PRO11', 'Nescafe’s Hazelnut Cold Coffee', 'The rich fruity strawberry flavor is unmissable in the other variant.', 17, 239, 'Dairy', 'SUP1'], ['PRO14', 'Amul Haldi Doodh', 'its sweetness levels were

In [157]:
[f for f in range(1,11)]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]